In [42]:
# face verification with the VGGFace2 model
from matplotlib import pyplot
from PIL import Image
from numpy import asarray
from scipy.spatial.distance import cosine
from mtcnn.mtcnn import MTCNN
from keras_vggface.vggface import VGGFace
from keras_vggface.utils import preprocess_input

In [56]:
# extract a single face from a given photograph
def extract_face(filename, required_size=(224, 224)):
    # load image from file
    pixels = pyplot.imread(filename)
    # create the detector, using default weights
    detector = MTCNN()
    # detect faces in the image
    results = detector.detect_faces(pixels)
    # extract the bounding box from the first face
    print(results)
    x1, y1, width, height = results[0]['box']
    x2, y2 = x1 + width, y1 + height
    # extract the face
    face = pixels[y1:y2, x1:x2]
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
    return face_array

# load the photo and extract the face
pixels = extract_face('images/yaniz3.jpeg')
# plot the extracted face
pyplot.imshow(pixels)
# show the plot
pyplot.show()

[]


IndexError: list index out of range

In [44]:
# extract faces and calculate face embeddings for a list of photo files
def get_embeddings(filenames):
    # extract faces
    faces = [extract_face(f) for f in filenames]
    # convert into an array of samples
    samples = asarray(faces, 'float32')
    # prepare the face for the model, e.g. center pixels
    samples = preprocess_input(samples, version=2)
    # create a vggface model
    model = VGGFace(model='resnet50', include_top=False, input_shape=(224, 224, 3), pooling='avg')
    # perform prediction
    yhat = model.predict(samples)
    return yhat

In [45]:

# determine if a candidate face is a match for a known face
def is_match(known_embedding, candidate_embedding, thresh=0.5):
    # calculate distance between embeddings
    score = cosine(known_embedding, candidate_embedding)
    if score <= thresh:
        print('>face is a Match (%.3f <= %.3f)' % (score, thresh))
    else:
        print('>face is NOT a Match (%.3f > %.3f)' % (score, thresh))

In [52]:
from os import listdir

mypath='images'
onlyfiles = [f for f in listdir(mypath)]
onlyfiles

['yaniz2.jpeg',
 'benja.jpeg',
 'yaniz4.jpeg',
 'yaniz3.jpeg',
 'chanchus.jpeg',
 'sharon_stone1.jpg']

In [53]:

# define filenames
filenames = [mypath+'/'+f for f in onlyfiles]
# get embeddings file filenames
embeddings = get_embeddings(filenames)
emb_map = {}
for i,f in enumerate(filenames):
    emb_map[f]=embeddings[i]
emb_map

IndexError: list index out of range

{'images/benja.jpeg': array([0.        , 1.1780062 , 9.251214  , ..., 0.40060043, 0.07547884,
        0.        ], dtype=float32),
 'images/chanchus.jpeg': array([7.1937885 , 1.5791302 , 0.        , ..., 0.1363132 , 1.8282299 ,
        0.08853175], dtype=float32),
 'images/sharon_stone1.jpg': array([0.17336304, 0.09939323, 0.        , ..., 7.3180475 , 6.4405637 ,
        2.005529  ], dtype=float32)}